<a href="https://colab.research.google.com/github/idhamari/differential_privacy_tutorials/blob/main/Differential_Privacy_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

In these tutorial, I will try to explain Differential Privacy (DP) in a practical way with examples using simple python scripts. I will try to add more details, examples, and scripts whenever i find the time, so please consider this work in progress.

**Differential Privacy** is a framework for measuring the privacy guarantees provided by an algorithm. Its central concept is a "privacy budget", a measure of how much a person's privacy is affected by participating in the dataset.

Let's start with the theoretical part:

**Randomized Algorithm:** An algorithm is randomized if its output contains a certain amount of randomness. In other words, given the same input, the output may not always be the same.

**Neighboring Databases:** We say that two databases, D1 and D2, are neighboring databases if they differ by only one entry. This difference corresponds to the information of exactly one individual.




## Epsilon-Differential Privacy (ϵ-DP):

A randomized algorithm A satisfies ϵ-DP if for all datasets D1 and D2 that differ on a single element, and all sets of possible outputs S, the following inequality holds:

Pr[A(D1) ∈ S] ≤ e^ϵ * Pr[A(D2) ∈ S]

This inequality means that the presence or absence of a single database item does not significantly change the probability distribution of the algorithm's output.

For simplicity, let's consider an example where we're trying to calculate the average age in a dataset, and we want to do it under differential privacy.


## Laplace Mechanism

Here's a Python implementation of **differential privacy using the Laplace Mechanism**, one of the most common techniques for adding noise. The Laplace mechanism adds noise proportional to the L1 sensitivity of the query / function, divided by the privacy parameter epsilon.

This code calculates the average age and adds noise to it. The amount of noise is determined by the sensitivity of the query (how much the result can change with the addition/removal of a single database entry) and the privacy parameter epsilon.

Keep in mind that this is a basic example. Differential privacy can be complex, especially when considering different types of queries, complex data types, or when you want to limit the cumulative privacy loss from multiple queries. These advanced topics often involve techniques such as composition theorems, privacy amplification, or post-processing. For further understanding, I recommend you to refer to detailed textbooks and academic papers on the subject.

In [3]:
import numpy as np

# Privacy parameter
epsilon = 0.5

# Our sensitive data
ages = np.array([25, 30, 35, 40, 45])

# The query we want to answer (average age)
query_result = np.mean(ages)

# Compute the L1 sensitivity of the query
sensitivity = 1.0 / len(ages)

# Add noise to the result of the query.
# The amount of noise is sampled from a Laplace distribution
noise = np.random.laplace(loc=0, scale=sensitivity/epsilon)

# The final, differentially private result
dp_result = query_result + noise

print(f"Query result: {query_result}")
print(f"Differentially private result: {dp_result}")


Query result: 35.0
Differentially private result: 35.28061752749318


## (ε, δ)-differential privacy

To handle the cumulative privacy loss from multiple queries, we introduce the concept of **(ε, δ)-differential privacy**. It extends ε-differential privacy by allowing the probability of a certain amount of additional privacy loss, governed by δ. A randomized algorithm is said to have (ε, δ)-differential privacy if it satisfies the ε-differential privacy condition with probability 1-δ.

The privacy parameter ε controls the 'worst-case' privacy loss, and δ allows the algorithm to have a very small chance of additional privacy loss. Normally, δ is set to be a very small positive number.

Let's extend our Python example by considering multiple queries, where each query is calculating the average age in a different way (e.g., average age of males and females). Here, we will simulate two queries:

In [4]:
import numpy as np

# Privacy parameters
epsilon = 0.5
delta = 0.01

# Our sensitive data (ages of males and females)
ages_males = np.array([25, 30, 35, 40, 45])
ages_females = np.array([27, 32, 37, 42, 47])

# Two queries we want to answer (average age for males and females)
query_result_males = np.mean(ages_males)
query_result_females = np.mean(ages_females)

# Compute the L1 sensitivity of the queries
sensitivity_males = 1.0 / len(ages_males)
sensitivity_females = 1.0 / len(ages_females)

# Add noise to the result of the queries.
# The amount of noise is sampled from a Laplace distribution
noise_males = np.random.laplace(loc=0, scale=sensitivity_males/epsilon)
noise_females = np.random.laplace(loc=0, scale=sensitivity_females/epsilon)

# The final, differentially private results
dp_result_males = query_result_males + noise_males
dp_result_females = query_result_females + noise_females

print(f"Query result for males: {query_result_males}")
print(f"Differentially private result for males: {dp_result_males}")
print(f"Query result for females: {query_result_females}")
print(f"Differentially private result for females: {dp_result_females}")


Query result for males: 35.0
Differentially private result for males: 35.40560720277377
Query result for females: 37.0
Differentially private result for females: 36.57699435712488


In the above Python script, we applied the Laplace Mechanism to each query independently. This is fine if we have a single privacy budget that we can split across all queries. However, the privacy guarantee degrades when the same individual's data is used in multiple queries.

To manage this, in practice, we use composition theorems to handle the cumulative privacy loss from multiple queries. Furthermore, more sophisticated mechanisms such as the Exponential Mechanism, Gaussian Mechanism, or advanced techniques like Differential Privacy under sampling, are used to handle different situations and provide better privacy-accuracy trade-offs.

Please note that applying differential privacy involves a deep understanding of data sensitivity, privacy requirements, and how the added noise can affect the usability of the data. Moreover, it's also important to understand how to set appropriate ε and δ values as per the privacy requirements. It's always recommended to refer to detailed textbooks and academic papers on differential privacy or consult with a privacy expert.

## Exponential Mechanism

Let's go a bit deeper into differential privacy. We'll discuss another type of mechanism for preserving privacy, known as the Exponential Mechanism.

**The Exponential Mechanism** is a differential privacy system that selects an output from a set according to a probability distribution. This probability is proportional to the exponential of the utility of the output divided by the privacy budget.

The utility of an output is a measure of how useful that output is. Higher utility means that the output is more desirable. By adding randomness in proportion to the utility, the Exponential Mechanism ensures that even if some outputs are more useful than others, an observer cannot be certain about the true data because all outputs have some probability of being chosen.

The Exponential Mechanism is often used when you need to select an output from a discrete set of outputs (like selecting the most common age group from a set of age groups).

Here is a simple implementation of the Exponential Mechanism in Python. In this example, we calculate the most common age group from a dataset. This code calculates the count of ages in each age group, adds noise to the counts using the Exponential Mechanism, and selects an age group according to the noisy counts. This gives us a differentially private way to find the most common age group.

In general, choosing the right differential privacy mechanism and setting the parameters requires understanding the data, the queries, and the privacy requirements. Advanced topics in differential privacy, such as the privacy accountant, moments accountant, and privacy amplification, are often used in practice to manage the privacy budget and provide stronger privacy guarantees.

Differential privacy is a complex field of study, and implementing it in a real-world scenario often requires expert knowledge and careful consideration. Always refer to the latest research and standards, and consider consulting with a privacy expert when necessary.

In [5]:
import numpy as np

# Privacy parameter
epsilon = 0.5

# Our sensitive data
ages = np.array([25, 30, 35, 40, 45, 25, 30, 25])

# Possible outputs
age_groups = np.array([20, 30, 40, 50])

# Calculate the utility of each output
# The utility is the count of the number of ages in each age group
utilities = np.array([np.sum((ages >= group) & (ages < group + 10)) for group in age_groups])

# Compute the sensitivity of the query
# In this case, it's 1, because adding or removing an individual can change the count by at most 1
sensitivity = 1.0

# Calculate the probabilities for each output
probabilities = np.exp(utilities * epsilon / (2 * sensitivity))

# Normalize the probabilities so they sum to 1
probabilities /= np.sum(probabilities)

# Choose an output according to the calculated probabilities
dp_result = np.random.choice(age_groups, p=probabilities)

print(f"Differentially private result: {dp_result}")


Differentially private result: 40


Now that you've seen two types of noise addition mechanisms for differential privacy (Laplace and Exponential), let's delve into a topic that is particularly important when you're performing multiple queries, which is the concept of the Privacy Budget.

The privacy budget is a limit on the total amount of information that can be revealed about an individual. In other words, the privacy budget quantifies the privacy loss that an individual is willing to tolerate. The privacy budget is often represented by the symbol ε. The smaller the value of ε, the smaller the privacy loss, and the greater the privacy protection.

The concept of privacy budget is important in the context of multiple queries. In differential privacy, every time you make a query against the data, some amount of privacy is 'spent' from the budget. If you make too many queries, you could exhaust the privacy budget, which means that the individuals in the dataset might no longer have their privacy protected.

The privacy budget is consumed according to the composition theorem. There are two types of composition: Sequential Composition and Parallel Composition.

Sequential Composition: If a series of k algorithms A1, A2, ..., Ak each satisfy ε-DP, then running them sequentially on the same dataset satisfies k*ε-DP. In other words, the privacy losses add up.

Parallel Composition: If a series of k algorithms each satisfy ε-DP, and they each operate on disjoint subsets of the data, then the whole sequence also satisfies ε-DP. This means that if each query operates on a different part of the data, the privacy loss does not add up.

These principles guide how to manage the privacy budget when making multiple queries.

Now, remember the noise addition strategies we discussed earlier? To ensure that your calculations remain within your privacy budget, you would typically add more noise for each subsequent query. However, how much noise to add can be a complex decision, as it can affect the utility of the results. There are different methods for calculating how much noise to add, often involving advanced mathematical and statistical concepts.

Overall, implementing differential privacy in a real-world scenario can be complex and requires careful consideration. Always refer to the latest research and standards, and consider consulting with a privacy expert when necessary.

## Advanced topics

You have already learned some key concepts about differential privacy, including privacy budget, sequential and parallel composition, the Laplace and Exponential Mechanisms, and ε-differential privacy and (ε, δ)-differential privacy. Now, let's talk about advanced topics such as Advanced Composition and Privacy Amplification.



### 1. Advanced Composition Theorems:

Advanced Composition Theorems help provide a more refined bound on the overall privacy loss when we conduct multiple computations or queries. The basic composition theorem would suggest that privacy loss compounds linearly with the number of computations, but in reality, the overall privacy loss is generally less than this linear sum.

Under the Advanced Composition Theorem, if we have k algorithms that each provide (ε, δ)-differential privacy, then running them sequentially would result in (ε√k log(1/δ), kδ)-differential privacy overall.



### 2. Privacy Amplification:

Privacy Amplification is the concept that privacy can be enhanced by randomness in the data selection process. In scenarios where we have random sampling, privacy amplification by sampling suggests that differential privacy can be strengthened simply by reducing the sample size. Privacy amplification allows the privacy parameter to scale with the size of the sample rather than the size of the whole database, which means you can obtain a better privacy-accuracy trade-off when you have a large database.



### 3. Post-processing:

Differential privacy has an important property called post-processing immunity, which means that any function applied to the output of a differentially private algorithm will also be differentially private. This property ensures that the privacy guarantee is preserved regardless of how the output is later used.

Implementing these concepts in Python would largely depend on the specific requirements of your application. Most of these concepts are abstract and don't directly translate into specific Python code, but they guide how you would structure your differential privacy algorithms and how much noise you should add.

Remember, differential privacy is a complex and active field of research. Implementing differential privacy in a real-world situation often requires careful analysis and sometimes even new theoretical developments. Always refer to the latest research and standards, and consider consulting with a privacy expert when necessary.